SPARK MLLIB.

Here I demonstrate several models of classifications on PAMAP Dataset. Refer to PAMAP_DataExploration.ipynb for contextual background of this dataset.

My aim is to predict the activity (running, walking, lying) using the features (heart rate, etc.) using Spark MLlib.
I demonstrate a binary classification algorithm using LogisticRegressionWithLBFGS, a multiclass classification using DecisionTree, and a linear regression using LinearRegressionSGD. Using linear regression to predict categorical label is obviously nonsensical, but I thought that it would be fun to just type that in anyway.

In [1]:
import numpy as np
import pandas as pd

In [2]:
#first we load the original .dat file into pandas dataframe
#this is the original file downloaded from UCI repository.

subject1 = pd.read_csv("protocol/subject101.dat", header=None, delimiter=r"\s+")
subject2 = pd.read_csv("protocol/subject102.dat", header=None, delimiter=r"\s+")
subject3 = pd.read_csv("protocol/subject103.dat", header=None, delimiter=r"\s+")
subject4 = pd.read_csv("protocol/subject104.dat", header=None, delimiter=r"\s+")
subject5 = pd.read_csv("protocol/subject105.dat", header=None, delimiter=r"\s+")
subject6 = pd.read_csv("protocol/subject106.dat", header=None, delimiter=r"\s+")
subject7 = pd.read_csv("protocol/subject107.dat", header=None, delimiter=r"\s+")
subject8 = pd.read_csv("protocol/subject108.dat", header=None, delimiter=r"\s+")
subject9 = pd.read_csv("protocol/subject109.dat", header=None, delimiter=r"\s+")

In [3]:
#drop all NaN values

subject1.dropna(axis=0, how='any', inplace=True)
subject2.dropna(axis=0, how='any', inplace=True)
subject3.dropna(axis=0, how='any', inplace=True)
subject4.dropna(axis=0, how='any', inplace=True)
subject5.dropna(axis=0, how='any', inplace=True)
subject6.dropna(axis=0, how='any', inplace=True)
subject7.dropna(axis=0, how='any', inplace=True)
subject8.dropna(axis=0, how='any', inplace=True)
subject9.dropna(axis=0, how='any', inplace=True)

In [6]:
#drop first column-the timestamp. We don't need it.

subject1.drop([0],1, inplace=True)
subject2.drop([0],1, inplace=True)
subject3.drop([0],1, inplace=True)
subject4.drop([0],1, inplace=True)
subject5.drop([0],1, inplace=True)
subject6.drop([0],1, inplace=True)
subject7.drop([0],1, inplace=True)
subject8.drop([0],1, inplace=True)
subject9.drop([0],1, inplace=True)

In [8]:
subject1.drop([0],1, inplace=True)
subject1.describe()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
count,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,...,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.000000,34090.0,34090.0,34090.0,34090.0
mean,5.497008,123.997683,32.433857,-3.340384,6.264125,3.415962,-3.280725,6.285456,3.581538,0.032837,...,-0.001820,0.001907,0.006607,-50.860565,-4.103442,10.414006,1.0,0.0,0.0,0.0
std,6.586998,23.939563,1.234277,6.347021,5.120173,3.377958,6.383645,5.127563,3.356070,1.068595,...,0.992148,0.599676,1.769829,25.771224,30.623882,29.556970,0.0,0.0,0.0,0.0
min,0.000000,78.000000,30.000000,-42.770700,-22.363700,-34.124800,-42.651600,-22.103900,-18.511800,-9.436280,...,-9.326890,-7.324840,-11.539700,-172.236000,-135.604000,-100.864000,1.0,0.0,0.0,0.0
25%,0.000000,104.000000,31.437500,-7.835380,3.591960,1.516190,-7.696230,3.637472,1.701705,-0.345442,...,-0.184936,-0.047529,-0.306461,-63.492525,-28.464775,-9.883985,1.0,0.0,0.0,0.0
50%,3.000000,125.000000,32.812500,-2.979140,6.001390,3.467760,-2.923060,6.047090,3.680110,-0.000512,...,-0.001024,0.003992,-0.002775,-53.273200,-5.111945,11.077950,1.0,0.0,0.0,0.0
75%,7.000000,136.000000,33.500000,1.668800,8.626678,5.548102,1.765718,8.604370,5.734387,0.341274,...,0.069334,0.166022,0.055884,-36.182525,20.683500,33.137350,1.0,0.0,0.0,0.0
max,24.000000,183.000000,33.875000,13.765000,92.053000,65.302700,13.607200,62.030900,50.842500,15.961900,...,9.068100,6.410380,9.263240,87.115200,121.599000,121.229000,1.0,0.0,0.0,0.0


In [9]:
subject3.head()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
0,0,81.0,29.5000,-0.232001,9.79451,-1.03558,-0.166991,9.81824,-1.23563,-0.024521,...,-0.006815,-0.012675,0.007382,-12.1093,-8.37471,9.85938,-0.253628,0.690605,-0.224148,0.639136
11,0,81.0,29.5000,-0.238416,9.64142,-1.18911,-0.273003,9.65298,-1.12945,0.017643,...,0.031211,0.001704,0.033612,-11.8881,-8.37762,9.48621,0.254919,-0.687894,0.227817,-0.640248
22,0,81.0,29.5000,-0.388241,9.83228,-1.19200,-0.272248,9.65306,-1.03885,-0.020617,...,-0.005203,0.010844,0.008394,-11.5599,-7.94467,9.24058,0.252766,-0.688024,0.225957,-0.641620
33,0,81.0,29.5625,-0.170060,9.75264,-1.41939,-0.319222,9.63811,-1.23508,0.009333,...,0.023685,-0.018569,0.013709,-11.2155,-8.18552,9.85409,0.252533,-0.688223,0.228102,-0.640738
44,0,81.0,29.5625,-0.169794,9.79058,-1.41951,-0.182445,9.74287,-1.20525,0.038800,...,0.027060,-0.020234,0.020792,-11.5503,-8.28406,9.85578,0.253148,-0.688337,0.227862,-0.640459


In [13]:
#as we have seen in the PAMAP_DataExploration.ipynb, out of all 24 categories, only about 8 are performed by all subjects
#Let us filter only activities 1,2,3,4,5,6,16,17 from subject 1
#we will use subject 1 for linear regression

subject1[subject1[1].isin([1,2,3,4,5,6,16,17])]

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
2932,1,100.0,30.3750,2.301060,7.25857,6.092590,2.207200,7.24042,5.955550,-0.069961,...,0.001351,-0.048878,-0.006328,-60.2040,-37.12250,-57.88470,1.0,0.0,0.0,0.0
2943,1,100.0,30.3750,2.246150,7.48180,5.552190,2.251300,7.40595,5.653130,-0.431227,...,0.005819,-0.017912,0.015243,-60.7820,-36.21160,-58.48910,1.0,0.0,0.0,0.0
2954,1,100.0,30.3750,2.300000,7.10681,6.093090,2.346130,7.46611,6.121030,0.075692,...,0.015376,-0.011798,0.011387,-61.4531,-36.29830,-58.48450,1.0,0.0,0.0,0.0
2965,1,100.0,30.3750,2.494550,7.52335,6.171570,2.591060,7.76653,6.271120,-0.259058,...,-0.012672,-0.009233,0.005823,-62.0065,-36.72170,-58.36080,1.0,0.0,0.0,0.0
2976,1,101.0,30.3750,2.716540,8.30596,4.786710,2.595960,7.94641,4.866350,0.377115,...,0.050959,-0.042562,-0.009289,-60.4311,-37.54120,-59.00130,1.0,0.0,0.0,0.0
2987,1,101.0,30.3750,2.549540,7.63122,5.556230,2.465200,7.61596,5.788360,-0.487667,...,0.005473,-0.003154,0.025621,-60.5521,-36.55220,-58.37080,1.0,0.0,0.0,0.0
2998,1,101.0,30.3750,2.824070,6.14490,5.065020,3.136310,6.11492,4.643300,-0.781563,...,0.011453,-0.000398,0.001243,-61.1082,-36.43210,-57.62230,1.0,0.0,0.0,0.0
3009,1,101.0,30.3750,2.736260,7.94195,6.520170,2.645260,8.56681,6.541130,-0.472414,...,0.003170,-0.019596,0.001829,-61.7855,-36.29310,-57.98670,1.0,0.0,0.0,0.0
3020,1,101.0,30.3750,2.388940,7.48830,6.401030,2.527590,7.32908,6.332550,0.358279,...,0.008459,-0.069819,0.032094,-61.5679,-35.96660,-58.23250,1.0,0.0,0.0,0.0
3031,1,101.0,30.3750,1.813200,6.85639,7.356720,1.505500,6.67288,7.331680,0.360442,...,-0.014122,-0.027863,0.006432,-62.2406,-35.94290,-58.22680,1.0,0.0,0.0,0.0


In [15]:
#we can apply the filter on all subjects and save it, in case if we are interested to work on larger datasets, time permitting.

subject_1 = subject1[subject1[1].isin([1,2,3,4,5,6,16,17])]
subject_2 = subject2[subject2[1].isin([1,2,3,4,5,6,16,17])]
subject_3 = subject3[subject3[1].isin([1,2,3,4,5,6,16,17])]
subject_4 = subject4[subject4[1].isin([1,2,3,4,5,6,16,17])]
subject_5 = subject5[subject5[1].isin([1,2,3,4,5,6,16,17])]
subject_6 = subject6[subject6[1].isin([1,2,3,4,5,6,16,17])]
subject_7 = subject7[subject7[1].isin([1,2,3,4,5,6,16,17])]
subject_8 = subject8[subject8[1].isin([1,2,3,4,5,6,16,17])]

In [18]:
#export df as text file so we can load it into LabeledPoint RDD easily (I'm cheating a bit here hehe....)
#there should be an easy way to convert Pandas df directly to Spark df. 

subject_1.to_csv(r'filtered/subject_1.txt', header=None, index=None, sep=",")
subject_2.to_csv(r'filtered/subject_2.txt', header=None, index=None, sep=",")
subject_3.to_csv(r'filtered/subject_3.txt', header=None, index=None, sep=",")
subject_4.to_csv(r'filtered/subject_4.txt', header=None, index=None, sep=",")
subject_5.to_csv(r'filtered/subject_5.txt', header=None, index=None, sep=",")
subject_6.to_csv(r'filtered/subject_6.txt', header=None, index=None, sep=",")
subject_7.to_csv(r'filtered/subject_7.txt', header=None, index=None, sep=",")
subject_8.to_csv(r'filtered/subject_8.txt', header=None, index=None, sep=",")

In [88]:
#we shall extract another dataset for binary classification
#for this, we will take only activity 1-lying and 5-running
#let us use subject 2 for this


lyingrunning_2 = subject2[subject2[1].isin([1,5])]

In [216]:
# I realize that it is actually impossible to do classification if the label is not binary (0 and 1)
# It is easier if we change the label in pandas before we load it to rdd (also because I am somewhat familiar with pd)

#let's do it with subject2

lyingrunning_2.loc[:,1].replace([1,5],[0,1], inplace=True)

#it took me forever to figure this out!!!

C:\Users\taufik.azri\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [218]:
lyingrunning_2.tail()

,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
400158,1,160.0,28.8125,-9.06818,-8.60990,-4.800640,-9.39810,-7.93848,-4.041330,-0.883650,...,-0.035779,1.831070,-3.34299,-30.2569,-18.64920,-7.554590,0.533488,-0.398956,0.532623,-0.522051
400169,1,160.0,28.8125,-15.22380,9.76276,3.435420,-12.43550,5.10701,1.701690,0.955873,...,-1.610540,0.560602,-3.75764,-12.7535,-28.84720,-8.920530,0.676097,-0.475242,0.349244,-0.441663
400180,1,160.0,28.8125,-2.91379,15.47930,4.331790,-3.70496,18.24670,5.182450,0.938203,...,0.017448,-2.540870,4.02956,-12.9557,-25.13650,-13.706700,0.716113,-0.420604,0.380342,-0.406958
400191,1,160.0,28.8125,5.60030,6.75108,-1.053420,5.24150,7.39339,-0.858331,0.780722,...,-0.553169,-4.311990,7.51846,-38.9986,-6.95378,-0.921848,0.416553,-0.326559,0.669944,-0.520593
400202,1,160.0,28.8125,7.82526,6.51524,-0.018621,7.64982,6.39494,-0.224375,1.339540,...,3.899170,0.146933,-2.08410,-41.7433,1.31546,6.863090,0.239028,-0.312853,0.679118,-0.619506


In [219]:
#save it to csv

lyingrunning_2.to_csv(r'binaryClassification/lyingrunning_2.txt', header=None, index=None, sep=",")

In [256]:
#let us use subject 3 for multiclass classification

multiactivities_3 = subject3[subject3[1].isin([1,2,3,4,5,6,16,17])]

In [257]:
#let us make sure these labels exist multiactivities_3

multiactivities_3[1].unique()

array([ 1,  2,  3, 17, 16,  4], dtype=int64)

In [258]:
#looks like only activities 1,2,3,4,16,and 17 exists in subject 3. That's okay. 
#encode them to 0-5
multiactivities_3.loc[:,1].replace([1,2,3,4,16,17],[0,1,2,3,4,5], inplace=True)

C:\Users\taufik.azri\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [259]:
multiactivities_3[1].unique()

array([0, 1, 2, 5, 4, 3], dtype=int64)

In [260]:
multiactivities_3.to_csv(r'multiclassClassification/multiactivities_3.txt', header=None, index=None, sep=",")

In [25]:
#We need a spar session to load the text file into Spark DataFrame

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

spark = SparkSession \
    .builder \
    .appName("Classification") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [105]:
#let us use the first subject in our model.
#at this point, we are working with Spark's DataFrame

df1 = spark.read.load("filtered/subject_1.txt", format="csv")

In [106]:
df1.show(1)

#this looks messy but it essentially shows that the data are nicely presented with a header for each column.

+---+-----+------+------------------+-------+-------+------+-----------------+-------+--------------------+----------+----------+-----------------+--------+--------+----+----+----+----+-------+--------+-------+--------+--------+-------+--------+-----------+----------+----------+-------------------+-------------------+-------+----+----+----+----+-----+-----------------+--------+---------+-------+-------+--------+----------+---------+-----------+------------------+--------+--------+----+----+----+----+
|_c0|  _c1|   _c2|               _c3|    _c4|    _c5|   _c6|              _c7|    _c8|                 _c9|      _c10|      _c11|             _c12|    _c13|    _c14|_c15|_c16|_c17|_c18|   _c19|    _c20|   _c21|    _c22|    _c23|   _c24|    _c25|       _c26|      _c27|      _c28|               _c29|               _c30|   _c31|_c32|_c33|_c34|_c35| _c36|             _c37|    _c38|     _c39|   _c40|   _c41|    _c42|      _c43|     _c44|       _c45|              _c46|    _c47|    _c48|_c49|_c50

In [107]:
#Create LabeledPoint
#LabeledPoint is an array that consists of the label and the features to be used in machine learning model.

from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *

#apparently df.map no longer works after spark 2.0. We must explicityly call df.rdd.map

temp = df1.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))

In [108]:
#let us do some inspection. 
#Notice that the numeric features are not scaled. 

temp.take(2)

[LabeledPoint(1.0, [100.0,30.375,2.30106,7.25857,6.09259,2.2072,7.24042,5.95555,-0.0699614,-0.0183275,0.00458168,9.15626,-67.1825,-20.0857,1.0,0.0,0.0,0.0,32.1875,0.352225,9.72437,-1.68665,0.173568,9.69512,-1.53414,0.000274589,-0.0133517,-0.0393151,0.462317,-50.711,42.9228,1.0,0.0,0.0,0.0,30.75,9.77578,-1.88582,0.0957749,9.64677,-1.7524,0.295902,0.00135137,-0.048878,-0.00632768,-60.204,-37.1225,-57.8847,1.0,0.0,0.0,0.0]),
 LabeledPoint(1.0, [100.0,30.375,2.24615,7.4818,5.55219,2.2513,7.40595,5.65313,-0.431227,0.00268486,-0.0629637,9.14612,-67.3936,-20.5508,1.0,0.0,0.0,0.0,32.1875,0.125825,9.7256,-1.61331,0.143761,9.71023,-1.50369,-0.0294109,-0.0373207,-0.0338049,1.11344,-50.1898,44.6495,1.0,0.0,0.0,0.0,30.75,9.73536,-2.03681,0.0191184,9.63201,-1.72208,0.356423,0.00581943,-0.0179122,0.0152434,-60.782,-36.2116,-58.4891,1.0,0.0,0.0,0.0])]

In [109]:
#Scaling

from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

features = df1.rdd.map(lambda row: row[1:])

In [110]:
features.take(1)

[('100.0',
  '30.375',
  '2.3010599999999997',
  '7.25857',
  '6.09259',
  '2.2072',
  '7.240419999999999',
  '5.95555',
  '-0.06996139999999999',
  '-0.0183275',
  '0.00458168',
  '9.156260000000001',
  '-67.1825',
  '-20.0857',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '32.1875',
  '0.352225',
  '9.72437',
  '-1.68665',
  '0.173568',
  '9.69512',
  '-1.53414',
  '0.000274589',
  '-0.0133517',
  '-0.0393151',
  '0.46231700000000003',
  '-50.711000000000006',
  '42.9228',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '30.75',
  '9.775780000000001',
  '-1.88582',
  '0.0957749',
  '9.64677',
  '-1.7524',
  '0.295902',
  '0.00135137',
  '-0.048878',
  '-0.00632768',
  '-60.20399999999999',
  '-37.1225',
  '-57.8847',
  '1.0',
  '0.0',
  '0.0',
  '0.0')]

In [112]:
standardizer = StandardScaler()
model = standardizer.fit(features)
features_transformed = model.transform(features)

features_transformed.take(1)

[DenseVector([4.1812, 29.7359, 0.3533, 1.1816, 1.6431, 0.3364, 1.1633, 1.6058, -0.0743, -0.0201, 0.0027, 0.2684, -2.1247, -0.7021, 0.0, 0.0, 0.0, 0.0, 24.9592, 0.1681, 2.026, -0.3214, 0.0834, 2.0151, -0.2929, 0.0006, -0.0231, -0.1166, 0.0194, -1.8906, 1.1813, 0.0, 0.0, 0.0, 0.0, 31.4309, 1.4739, -0.2494, 0.0248, 1.5533, -0.2599, 0.0808, 0.0017, -0.0868, -0.0036, -2.1349, -1.1613, -1.8482, 0.0, 0.0, 0.0, 0.0])]

In [113]:
#putting back labels and features
#lets separate the label first

labels = df1.rdd.map(lambda row: int(row[0]))
labels.take(1)

[1]

In [62]:
labels.count()

16840

In [63]:
#making sure we have eight label as intended

labels.distinct().count()

8

In [114]:
transformedData = labels.zip(features_transformed)
transformedData.take(1)

[(1,
  DenseVector([4.1812, 29.7359, 0.3533, 1.1816, 1.6431, 0.3364, 1.1633, 1.6058, -0.0743, -0.0201, 0.0027, 0.2684, -2.1247, -0.7021, 0.0, 0.0, 0.0, 0.0, 24.9592, 0.1681, 2.026, -0.3214, 0.0834, 2.0151, -0.2929, 0.0006, -0.0231, -0.1166, 0.0194, -1.8906, 1.1813, 0.0, 0.0, 0.0, 0.0, 31.4309, 1.4739, -0.2494, 0.0248, 1.5533, -0.2599, 0.0808, 0.0017, -0.0868, -0.0036, -2.1349, -1.1613, -1.8482, 0.0, 0.0, 0.0, 0.0]))]

In [115]:
# We need to put the labels and features as a LabeledPoint rdd before passing it to MLlib.

transformedData = transformedData.map(lambda row: LabeledPoint(row[0],[row[1]]))
#transformedData.take(1)

In [127]:
type(transformedData)

#an RDD as expected

pyspark.rdd.PipelinedRDD

In [116]:
#train-test split 

trainingData, testingData = transformedData.randomSplit([.8,.2],seed=65)

In [117]:
from pyspark.mllib.regression import LinearRegressionWithSGD

lrm = LinearRegressionWithSGD.train(trainingData, iterations=10)

C:\Spark\python\pyspark\mllib\regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [119]:
lrm.weights

DenseVector([-2.8708, -1.9065, 3.0910, -5.6809, -6.5451, 3.0244, -5.6577, -6.8022, -2.0803, -2.5813, -1.1830, -2.2250, 5.0147, 6.3955, 0.0, 0.0, 0.0, 0.0, -1.6296, -1.0585, -9.1995, 2.7719, -5.8076, -9.2106, 2.4750, -6.2808, -4.6230, 3.6399, -4.3373, 9.2663, -1.0400, 0.0, 0.0, 0.0, 0.0, -2.0572, -7.8520, -2.6392, 4.1944, -8.3324, -3.9311, 3.7756, 1.7253, -1.3111, -6.4743, 1.0536, 2.1400, -2.5694, 0.0, 0.0, 0.0, 0.0])

In [120]:
testingData.take(1)

[LabeledPoint(1.0, [4.22300456179,29.7358750784,0.366775360108,1.218996935,1.72623500329,0.385249747781,1.17757541874,1.70739840829,0.380316375965,1.14817623305,0.200746000553,0.354788928989,-1.96775019717,-1.05329511907,0.0,0.0,0.0,0.0,25.0076329517,0.151304169948,1.98692862673,-0.299278717271,0.141280422717,1.99935091912,-0.275811608976,-0.0352264558621,-0.0376018939886,-0.101976930944,0.0576414733725,-1.90901834127,1.2289252212,0.0,0.0,0.0,0.0,31.4308561403,1.4776109975,-0.243924505508,-0.0950400364089,1.55081346242,-0.275593696178,0.0726918775242,0.0109245915843,-0.123956386561,0.0182119716223,-2.18325285097,-1.1251210512,-1.85928009329,0.0,0.0,0.0,0.0])]

In [129]:
lrm.predict([4.22300456179,29.7358750784,0.366775360108,1.218996935,1.72623500329,0.385249747781,1.17757541874,1.70739840829,0.380316375965,1.14817623305,0.200746000553,0.354788928989,-1.96775019717,-1.05329511907,0.0,0.0,0.0,0.0,25.0076329517,0.151304169948,1.98692862673,-0.299278717271,0.141280422717,1.99935091912,-0.275811608976,-0.0352264558621,-0.0376018939886,-0.101976930944,0.0576414733725,-1.90901834127,1.2289252212,0.0,0.0,0.0,0.0,31.4308561403,1.4776109975,-0.243924505508,-0.0950400364089,1.55081346242,-0.275593696178,0.0726918775242,0.0109245915843,-0.123956386561,0.0182119716223,-2.18325285097,-1.1251210512,-1.85928009329,0.0,0.0,0.0,0.0])

-1.648425562341093e+32

In [146]:
#if we want to pass the entire features to predict function, we first need to extract the feature array from the LabeledPoint
#remember LabeledPoint is consists of two  things, a label and an array of features

features_on_data = trainingData.map(lambda row: row.features[0])

In [147]:
features_on_data.collect()

[array([  4.18119264e+00,   2.97358751e+01,   3.53283092e-01,
          1.18159991e+00,   1.64305465e+00,   3.36416604e-01,
          1.16333027e+00,   1.60575070e+00,  -7.42646544e-02,
         -2.01060566e-02,   2.65668200e-03,   2.68401153e-01,
         -2.12474047e+00,  -7.02060428e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.49591685e+01,   1.68094269e-01,   2.02596510e+00,
         -3.21408482e-01,   8.34189816e-02,   2.01511832e+00,
         -2.92945647e-01,   5.99911763e-04,  -2.31343091e-02,
         -1.16598998e-01,   1.93608296e-02,  -1.89062694e+00,
          1.18133103e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   3.14308561e+01,
          1.47392911e+00,  -2.49425349e-01,   2.47815185e-02,
          1.55325764e+00,  -2.59885484e-01,   8.08473197e-02,
          1.74531494e-03,  -8.67779392e-02,  -3.59071000e-03,
         -2.13488774e+00,  -1.16128036e+00,  -1.84817534e+00,
        

In [150]:
#we can also do the same thing to get the label, just for fun

labels_on_data = trainingData.map(lambda row: row.label)
labels_on_data.collect()

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [152]:
labels_on_data.distinct().count()

#again, just checking

8

In [144]:
#Model Evaluation

#first we get a tuple RDD that is a prediction and an actual values, which we will passed later to regression metrics
#we will use map function on training data to obtain these two values

from pyspark.mllib.evaluation import RegressionMetrics

metricsRDD = trainingData.map(lambda row: (
        float(lrm.predict(row.features[0]))
        ,row.label))

In [145]:
metricsRDD.collect()

[(-1.6471126296139064e+32, 1.0),
 (-1.6469884790110517e+32, 1.0),
 (-1.6472014896446046e+32, 1.0),
 (-1.647284907318494e+32, 1.0),
 (-1.646920703661134e+32, 1.0),
 (-1.6479559068891884e+32, 1.0),
 (-1.647426174904952e+32, 1.0),
 (-1.6485109975854507e+32, 1.0),
 (-1.648719491826622e+32, 1.0),
 (-1.6484753716523404e+32, 1.0),
 (-1.6484156984903467e+32, 1.0),
 (-1.6479522292764563e+32, 1.0),
 (-1.647645507337588e+32, 1.0),
 (-1.6502386169865587e+32, 1.0),
 (-1.6495064947047474e+32, 1.0),
 (-1.6495908061935058e+32, 1.0),
 (-1.6495570545202675e+32, 1.0),
 (-1.649512850926296e+32, 1.0),
 (-1.649330621866265e+32, 1.0),
 (-1.649108234448914e+32, 1.0),
 (-1.6489002749718143e+32, 1.0),
 (-1.6486636126827966e+32, 1.0),
 (-1.6488665774080098e+32, 1.0),
 (-1.6489624144744986e+32, 1.0),
 (-1.6484243996128615e+32, 1.0),
 (-1.649257204071764e+32, 1.0),
 (-1.6498785746649837e+32, 1.0),
 (-1.648236369603297e+32, 1.0),
 (-1.64821129561248e+32, 1.0),
 (-1.6484459201632764e+32, 1.0),
 (-1.649827101542218e+

In [134]:
metrics = RegressionMetrics(metricsRDD)

#the result is going to be terrible, trust me.

In [135]:
metrics.r2

-9.116810886273252e+62

In [154]:
metrics.rootMeanSquaredError

1.7758403981832268e+32

In [220]:
#BINARY CLASSIFICATION

#load data
#let us use the second subject in our model, where we have filtered only activities 1 and 5.

df2 = spark.read.load("binaryClassification/lyingrunning_2.txt", format="csv")

In [221]:
df2.show(1)

+---+----+-------+-------+------------------+-------+------------------+------------------+-------+----------+----------+----------+-------+--------+-------+--------+--------+---------+---------+-------+--------+------+--------+--------+-------+--------+---------+--------------------+--------------------+--------+--------+-------+---------+-----------+--------+--------+-----+-------+--------+--------+-----------------+--------+--------+---------+----------+-------------------+--------+------------------+-------+----------+-------+--------------------+--------+
|_c0| _c1|    _c2|    _c3|               _c4|    _c5|               _c6|               _c7|    _c8|       _c9|      _c10|      _c11|   _c12|    _c13|   _c14|    _c15|    _c16|     _c17|     _c18|   _c19|    _c20|  _c21|    _c22|    _c23|   _c24|    _c25|     _c26|                _c27|                _c28|    _c29|    _c30|   _c31|     _c32|       _c33|    _c34|    _c35| _c36|   _c37|    _c38|    _c39|             _c40|    _c41|  

In [222]:
#scaling

features2 = df2.rdd.map(lambda row: row[1:])

standardizer = StandardScaler()
model2 = standardizer.fit(features2)
features2_transformed = model2.transform(features2)

features2_transformed.take(1)

[DenseVector([3.5371, 15.6341, -0.9829, 0.9203, 0.811, -0.9457, 0.8998, 0.8222, 0.0068, -0.0329, -0.0409, 1.6229, -2.5225, -1.3779, 0.6551, 2.1616, -1.3684, -2.1756, 17.5783, 0.4866, 2.1016, -0.4608, 0.3803, 2.0829, -0.3797, 0.0603, -0.0358, -0.0305, -1.2483, -2.4309, 0.4575, 0.0641, -0.0393, 2.1378, 3.2339, 27.6982, 1.0506, -0.1397, -0.4927, 1.1991, -0.158, -0.3931, 0.0148, -0.0202, 0.0193, -3.6138, 0.0315, 4.3091, 0.0622, 2.041, -0.3539, 1.6919])]

In [223]:
#making sure the label looks good and it is BINARY

labels2 = df2.rdd.map(lambda row: int(row[0]))
labels2.count()

2920

In [224]:
labels2.distinct().count()

2

In [167]:
#labels have to be 0 or 1. Let's encode it.

#screw this

#tempLabels = labels2.map(lambda x: )

In [226]:
labels2.distinct().collect()

[0, 1]

In [227]:
transformedData2 = labels2.zip(features2_transformed)
transformedData2.take(1)

[(0,
  DenseVector([3.5371, 15.6341, -0.9829, 0.9203, 0.811, -0.9457, 0.8998, 0.8222, 0.0068, -0.0329, -0.0409, 1.6229, -2.5225, -1.3779, 0.6551, 2.1616, -1.3684, -2.1756, 17.5783, 0.4866, 2.1016, -0.4608, 0.3803, 2.0829, -0.3797, 0.0603, -0.0358, -0.0305, -1.2483, -2.4309, 0.4575, 0.0641, -0.0393, 2.1378, 3.2339, 27.6982, 1.0506, -0.1397, -0.4927, 1.1991, -0.158, -0.3931, 0.0148, -0.0202, 0.0193, -3.6138, 0.0315, 4.3091, 0.0622, 2.041, -0.3539, 1.6919]))]

In [228]:
#put labels and features together as LabeledPoint

transformedData2 = transformedData2.map(lambda row: LabeledPoint(row[0],[row[1]]))
transformedData2.take(1)

[LabeledPoint(0.0, [3.53714774704,15.6341011046,-0.982882867735,0.920264035478,0.811041234792,-0.945675896673,0.899842663997,0.82219784361,0.00681817207152,-0.032861782129,-0.0408767896708,1.62294312953,-2.52253240247,-1.37793625255,0.655102476295,2.16163379618,-1.36837608501,-2.17556717202,17.5782607547,0.486606464384,2.10155471656,-0.460808491618,0.380333098222,2.08293754841,-0.379651293895,0.0603407567104,-0.0358304747422,-0.0305144497136,-1.2482751397,-2.43091862545,0.457499050044,0.0640827404899,-0.039266513436,2.13783210831,3.23386587058,27.6981793157,1.05060511394,-0.139657953862,-0.492691880586,1.19912069778,-0.158030692186,-0.393120459204,0.0148422242558,-0.0202120552795,0.0192975024138,-3.61375117269,0.031523122632,4.30909126638,0.062211399692,2.04101158424,-0.353863479765,1.69189120901])]

In [229]:
#train-test split 

trainingData2, testingData2 = transformedData2.randomSplit([.8,.2],seed=121)

In [230]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

clf = LogisticRegressionWithLBFGS.train(trainingData2)

In [232]:
#extract features from testingData2 to be passed to predict

features_on_test2 = testingData2.map(lambda row: row.features[0])

In [233]:
clf.predict(features_on_test2)

PythonRDD[616] at RDD at PythonRDD.scala:48

In [234]:
#whoops don't forget to call collect

clf.predict(features_on_test2).collect()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [235]:
#Time for model evaluation

from pyspark.mllib.evaluation import BinaryClassificationMetrics

#similar to regression metrics, this one takes a tuple of predicted label (score) and actual labels(label)

scoreAndLabels = trainingData2.map(lambda row: (float(clf.predict(row.features[0])),row.label))
metrics2 = BinaryClassificationMetrics(scoreAndLabels)

In [236]:
metrics2.areaUnderROC

#the area under the receiver operating characteristic (ROC) curve

1.0

In [237]:
metrics2.areaUnderPR

#area under the precision-recall curve

1.0

In [239]:
#let's do the same thing for testing data

scoreAndLabels = testingData2.map(lambda row: (float(clf.predict(row.features[0])),row.label))
metrics2 = BinaryClassificationMetrics(scoreAndLabels)

print("Area under ROC: ", metrics2.areaUnderROC)
print("Area under PR: ", metrics2.areaUnderPR)

#WHOA thats's perfect.

Area under ROC:  1.0
Area under PR:  1.0


In [261]:
#MULTICLASS CLASSIFICATION

#Let's use DecisionTree and RandomForest
#we will load subject 3, where we have filtered only activities 1,2,3,4,5,6,16,17.

df3 = spark.read.load("multiclassClassification/multiactivities_3.txt", format="csv")
df3.take(1)

[Row(_c0='0', _c1='91.0', _c2='31.0', _c3='-2.31698', _c4='6.7534600000000005', _c5='9.18109', _c6='-2.1177599999999996', _c7='6.2323', _c8='8.86153', _c9='-0.14798599999999998', _c10='0.0701193', _c11='-0.133127', _c12='-13.0988', _c13='-22.8999', _c14='-37.6387', _c15='0.047148', _c16='-0.0745711', _c17='0.293823', _c18='0.951779', _c19='33.0625', _c20='-0.647602', _c21='9.70533', _c22='0.579833', _c23='-0.819408', _c24='9.7846', _c25='0.8599030000000001', _c26='0.0675537', _c27='-0.169576', _c28='0.0201355', _c29='-13.0368', _c30='-33.2082', _c31='-1.0186899999999999', _c32='0.0896736', _c33='-0.023400400000000002', _c34='-0.670596', _c35='-0.736011', _c36='31.125', _c37='9.89763', _c38='0.59308', _c39='-1.34617', _c40='9.71673', _c41='1.0137399999999999', _c42='-1.13608', _c43='-0.63722', _c44='-0.0742192', _c45='0.11306600000000001', _c46='-42.6776', _c47='0.5741229999999999', _c48='57.4883', _c49='0.69023', _c50='-0.011488799999999999', _c51='0.723498', _c52='0.000397506999999999

In [262]:
#At this point, I could have created a function to parse and load this data into RDD.

#scaling

features3 = df3.rdd.map(lambda row: row[1:])

standardizer = StandardScaler()
model3 = standardizer.fit(features3)
features3_transformed = model3.transform(features3)

features3_transformed.take(1)

[DenseVector([5.6547, 27.7555, -0.3598, 1.9801, 2.566, -0.3262, 1.823, 2.4682, -0.1179, 0.0674, -0.0915, -0.7672, -1.2803, -2.3464, 0.1627, -0.1674, 0.7881, 1.6656, 29.1446, -0.3885, 2.5745, 0.1277, -0.491, 2.5855, 0.1902, 0.3149, -0.5245, 0.0953, -0.9889, -2.3232, -0.0703, 0.3244, -0.0883, -1.7341, -2.0889, 32.8794, 2.1613, 0.1042, -0.4683, 2.1184, 0.178, -0.3976, -1.0497, -0.1806, 0.0808, -3.2247, 0.0372, 3.6956, 4.0205, -0.0185, 2.397, 0.0007])]

In [263]:
#making sure the label looks good and it is BINARY

labels3 = df3.rdd.map(lambda row: int(row[0]))
print(labels3.count())
print(labels3.distinct().collect())

13505
[0, 2, 4, 1, 3, 5]


In [264]:
transformedData3 = labels3.zip(features3_transformed)
transformedData3.take(1)

[(0,
  DenseVector([5.6547, 27.7555, -0.3598, 1.9801, 2.566, -0.3262, 1.823, 2.4682, -0.1179, 0.0674, -0.0915, -0.7672, -1.2803, -2.3464, 0.1627, -0.1674, 0.7881, 1.6656, 29.1446, -0.3885, 2.5745, 0.1277, -0.491, 2.5855, 0.1902, 0.3149, -0.5245, 0.0953, -0.9889, -2.3232, -0.0703, 0.3244, -0.0883, -1.7341, -2.0889, 32.8794, 2.1613, 0.1042, -0.4683, 2.1184, 0.178, -0.3976, -1.0497, -0.1806, 0.0808, -3.2247, 0.0372, 3.6956, 4.0205, -0.0185, 2.397, 0.0007]))]

In [265]:
#put labels and features together as LabeledPoint

transformedData3 = transformedData3.map(lambda row: LabeledPoint(row[0],[row[1]]))
transformedData3.take(1)

[LabeledPoint(0.0, [5.65467417904,27.7555062117,-0.359777416984,1.98005379536,2.5660022149,-0.326223876885,1.82301521699,2.46817920475,-0.117856461454,0.067430190853,-0.0915342316533,-0.767162161199,-1.28028524394,-2.3464412575,0.162688377724,-0.167383197031,0.78805834114,1.6656039799,29.1446196511,-0.388532114917,2.57450271678,0.127651943852,-0.491020747283,2.58548781926,0.190231817551,0.314883818112,-0.524482727552,0.0952707242853,-0.988900544113,-2.32322160424,-0.0703012373512,0.324428323706,-0.0883052529684,-1.73412710702,-2.08888111437,32.8793505356,2.1612988416,0.10419089675,-0.468306646066,2.11844162517,0.177983549174,-0.397578057373,-1.04970761715,-0.180559977111,0.0807666901808,-3.22470117302,0.0371836954066,3.6955824612,4.0204855189,-0.0184886633969,2.39703918135,0.000670941253367])]

In [267]:
#train-test split 

trainingData3, testingData3 = transformedData3.randomSplit([.8,.2],seed=212)

In [268]:
#DECISION TREE

from pyspark.mllib.tree import DecisionTree

tree = DecisionTree.trainClassifier(trainingData3, 6, {})

In [274]:
features_on_test3 = testingData3.map(lambda row: row.features[0])
tree.predict(features_on_test3).take(5)

[0.0, 0.0, 0.0, 0.0, 0.0]

In [282]:
predictions = tree.predict(testingData3.map(lambda x: x.features[0]))
labelsAndPredictions = testingData3.map(lambda lp: lp.label).zip(predictions)
#testErr = labelsAndPredictions.filter(lambda v, p: v != p).count() / float(testingData3.count())
testErr = labelsAndPredictions.filter(lambda item: item[0] != item[1]).count() / float(testingData3.count())
print('Test Error = ' + str(testErr))

Test Error = 0.008295625942684766


In [280]:
errorCount = labelsAndPredictions.filter(lambda item: item[0] != item[1]).count()

In [281]:
errorCount

22

In [277]:
print('Learned classification tree model:')
print(tree.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 15 nodes
  If (feature 0 <= 6.3382062226628575)
   If (feature 18 <= 31.017808815815766)
    If (feature 1 <= 28.2031756666784)
     Predict: 0.0
    Else (feature 1 > 28.2031756666784)
     Predict: 1.0
   Else (feature 18 > 31.017808815815766)
    If (feature 35 <= 35.850376186371214)
     If (feature 45 <= -1.9152991328955422)
      If (feature 47 <= 0.7464731623248169)
       Predict: 4.0
      Else (feature 47 > 0.7464731623248169)
       Predict: 5.0
     Else (feature 45 > -1.9152991328955422)
      If (feature 33 <= 0.20950428477557398)
       Predict: 1.0
      Else (feature 33 > 0.20950428477557398)
       Predict: 2.0
    Else (feature 35 > 35.850376186371214)
     Predict: 4.0
  Else (feature 0 > 6.3382062226628575)
   Predict: 3.0



In [283]:
#Model evaluation-multiclass regression metrics

from pyspark.mllib.evaluation import MulticlassMetrics

#multiclass metrics takes a tuple of predictionandlabels

predictionsAndLabels = tree.predict(testingData3.map(lambda x: x.features[0])).zip(testingData3.map(lambda y: y.label))
#predictionsAndLabels = testingData3.map(lambda lp: lp.label).zip(predictions)

metrics3 = MulticlassMetrics(predictionsAndLabels)



In [285]:
metrics3.confusionMatrix().toArray()

array([[ 396.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  518.,    2.,    0.,    0.,    0.],
       [   0.,    0.,  349.,    0.,    0.,    0.],
       [   0.,    0.,    0.,  513.,    0.,    0.],
       [   0.,    6.,    1.,    0.,  353.,    0.],
       [   0.,    0.,   13.,    0.,    0.,  501.]])

In [287]:
print("False positive rate for 0: ", metrics3.falsePositiveRate(0.0))
print("False positive rate for 1: ", metrics3.falsePositiveRate(1.0))
print("False positive rate for 2: ", metrics3.falsePositiveRate(2.0))
print("False positive rate for 3: ", metrics3.falsePositiveRate(3.0))
print("False positive rate for 4: ", metrics3.falsePositiveRate(4.0))
print("False positive rate for 5: ", metrics3.falsePositiveRate(5.0))

False positive rate for 0:  0.0
False positive rate for 1:  0.0028142589118198874
False positive rate for 2:  0.006947459834997829
False positive rate for 3:  0.0
False positive rate for 4:  0.0
False positive rate for 5:  0.0


In [288]:
print("Accuracy: ", metrics3.accuracy)

Accuracy:  0.9917043740573153


In [294]:
for i in range(6):
    print("precision %d: " %i, metrics3.precision(i))

precision 0:  1.0
precision 1:  0.9885496183206107
precision 2:  0.9561643835616438
precision 3:  1.0
precision 4:  1.0
precision 5:  1.0


In [295]:
for i in range(6):
    print("recall %d: " %i, metrics3.recall(i))

recall 0:  1.0
recall 1:  0.9961538461538462
recall 2:  1.0
recall 3:  1.0
recall 4:  0.9805555555555555
recall 5:  0.9747081712062257


In [301]:
for i in range(6):
    print("fMeasure %d: " %i, metrics3.fMeasure(float(i)))

fMeasure 0:  1.0
fMeasure 1:  0.992337164750958
fMeasure 2:  0.9775910364145658
fMeasure 3:  1.0
fMeasure 4:  0.9901823281907433
fMeasure 5:  0.9871921182266009


In [302]:
print("Weighted false positive rate: ", metrics3.weightedFalsePositiveRate)
print("Weighted precision: ", metrics3.weightedPrecision)
print("Weighted recall: ", metrics3.weightedRecall)

Weighted false positive rate:  0.0014660928041329503
Weighted precision:  0.9919861128920555
Weighted recall:  0.9917043740573152


In [304]:
print(metrics3.weightedFMeasure())

0.9917334028266676
